In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pprint
import pandas as pd
import random

I am implementing value iteration and policy iteration algorithms as given in Chapter 4 of **'Reinforcement Learning: An Introduction' by Sutton and Barto.**
(Figure 4.3:Policy Iteration and Figure 4.5:Value Iteration)

In [ ]:
##Initializing the states
## States are basically the towns
States = ['A', 'B', 'C']

In [ ]:
# Initializing actions 
Actions = {}
Actions['A'] = [1, 2, 3]
Actions['B'] = [1, 2]
Actions['C'] = [1, 2, 3]

In [6]:
# Implementing the reward function.
# Reward function g. 
Rewards = {}
Rewards['A'] = np.array([[10 , 4, 8], 
                        [8, 2, 4],
                        [4 , 6, 4]])
Rewards['B'] = np.array([[14 , 0, 18], 
                        [8, 16, 8]])

Rewards['C'] = np.array([[10 , 2, 8], 
                        [6, 4, 2],
                         [4, 0, 8]])
                         

def g (s, a, sf):
  return Rewards[s][a-1][ord(sf) - ord('A')]

 #Sanity check for reward function
print(g('A',3, 'B'))
print(g('C', 2, 'B'))


6
4


In [7]:
## Implementing probablity function

Prob = {}
Prob['A'] = np.array([[1/2 , 1/4, 1/4], 
                        [1/ 16, 3/4, 3/16],
                        [1/4 , 1/8, 5/8]])
Prob['B'] = np.array([[1/2 , 0, 1/2], 
                        [1/ 16, 7/8, 1/16]])

Prob['C'] = np.array([[1/4 , 1/4, 1/2], 
                        [1/ 8, 3/4, 1/8],
                        [3/4 , 1/16, 3/16]])

def p (s, a, sf):
  return Prob[s][a-1][ord(sf) - ord('A')]

#Sanity Check
print (p ('B', 2, 'B'))


0.875


I am modelling the problem as an infinite horizon problem therefore the policy only depends on the states.



The cell below implements value iteration as the function of beta (discount factor). 

I am going to implement value iteration. I am implementing the **Gauss-Seidel method (Asynchronous value iteration)** since it is more natural to implement when we are not doing any parallel processing. (Fig 4.5 Introduction to Reinforcement Learning, Sutton and Barto)

In [ ]:
def value_iteration(beta):
  ## Initializing v with zeros. (Here v is J basically) 
  v = {}
  v['A'] = 0
  v['B'] = 0
  v['C'] = 0
  ## Poilcy pi is also a dictionary (state : action)
  pi = {}

  while (True):
    delta = 0

    for s in States:
      
      prev_v = v[s]
      max_val = -183303 # -inf
      
      for a in Actions[s]:
          
          val = 0
          
          for sf in States:
            # print (s, a, sf)
            val += p(s, a, sf)*(g(s, a, sf) + beta * v[sf])
          
          if (val > max_val):
            max_val = val
            pi[s] = a
          
      v[s] = max_val
      delta = max(delta, abs(max_val - prev_v))

    # print(delta)
    if (delta == 0):
      return (v, pi)



In [ ]:
# value iteration for different betas
# And converting to dataframe for better tabulation

ValItResults = {}
piItResults = {}
for b in range (0, 96, 5):
  beta = b /100
  # print(beta ,value_iteration(beta))
  v, pi = value_iteration(beta)
  ValItResults[beta] =  v  
  piItResults[beta] = pi

df_J = pd.DataFrame(ValItResults).T


df_PI = pd.DataFrame(piItResults).T



In [10]:
df_J

,A,B,C
0.00,8.000000,16.000000,7.000000
0.05,8.511527,16.400260,7.498869
0.10,9.076506,16.856369,8.050865
0.15,9.708121,17.464503,8.669160
0.20,10.437030,18.482143,9.384398
0.25,11.274074,19.629630,10.207407
0.30,12.243837,20.934066,11.162756
0.35,13.378714,22.430769,12.282824
0.40,14.722222,24.166667,13.611111
0.45,16.334131,26.205534,15.207371


In [11]:
df_PI

,A,B,C
0.00,1,1,1
0.05,1,1,1
0.10,1,1,1
0.15,1,2,1
0.20,1,2,1
0.25,1,2,1
0.30,1,2,1
0.35,1,2,1
0.40,1,2,1
0.45,1,2,1


Now we will be implementing Q-Learning and TD-learning! For this we will write a function take_action(s, a) which will return the return the final state with given probabilities. We will also need a probabilistic function to implement epsilon greedy selection.`

In [12]:
def take_action (s, a):
  rnd = random.uniform(0, 1)
  cum_prob = 0;
  for sf in States:
    if (rnd >= cum_prob and rnd <= cum_prob + p(s, a, sf)):
      return (sf, g(s, a, sf) )
    else: 
      cum_prob += p(s, a, sf)

#Sanity Check
print(take_action('B', 1))

('A', 14)


In [ ]:
# Here we implement episilon_greedy
 epsilon = 0.05 # We can change this 

def epsilon_greedy (q, s):
  k = len(Actions[s])
  max_q = -13313 # -inf 
  A_max = 1
  for a in Actions[s]:
    if (q[s][a] > max_q):
      A_max = a
      max_q = q[s][a]
  rnd = random.uniform(0, 1)
  if (rnd < (1 - epsilon)):
    return A_max
  cum_prob = (1 - epsilon)
  for a in Actions[s]:
    if (rnd > cum_prob and rnd <= cum_prob + epsilon/k):
      return a
    else: 
      cum_prob += epsilon/k

  return A_max


  

The following cell implements q-learning (off policy) as given in Fig 6.12 (Introduction to Reinforcement Learning, Sutton and Barto).

In [ ]:
lr = 0.6 # Learning rate (it decreases with episodes)

def q_learning (beta):
  Q = {}
  #Initializing Q
  for s in States:
    Q[s] = {}
    for a in Actions[s]:
      # Q[s][a] = random.uniform(0, 1)
      Q[s][a] = 0

  for i in range(1000): #100 episodes
    S = random.choice(States)
    alpha = lr * (0.85)**(i//50) #LR scheduling
    for j in range(1000): #Since we don't have a termination state we go for finite steps.
      A = epsilon_greedy(Q, S)
      (Sf, R) = take_action (S, A)
      max_q = - 73931 # -inf
      for a in Actions[Sf]:
        max_q = max(max_q, Q[Sf][a])
      
      Q[S][A] = Q[S][A] + alpha*(R + beta*max_q - Q[S][A]) # This is the update
      S = Sf

  return Q

  



In [15]:
print (q_learning(0.95))

{'A': {1: 251.9349089937986, 2: 256.4919152980042, 3: 247.40576950081248}, 'B': {1: 259.3075438839347, 2: 270.2676827618121}, 'C': {1: 251.57693914712314, 2: 258.078284405911, 3: 246.32395574382494}}


In [ ]:
#Calculating as required by the problme
Q_his = {}
PI_q_his = {}
for b in range (0, 96, 5):
  beta = b /100
  Q = q_learning(beta)
  q_rearranged = {}
  PI_Q = {}
  for s in Q.keys():
    PI_Q[s] = max(Q[s] , key=Q[s].get)
    for a in Q[s].keys():
      q_rearranged[(s, a)] = Q[s][a]
      
  Q_his[beta] = q_rearranged
  PI_q_his[beta] = PI_Q



df_Q = pd.DataFrame(Q_his).T
df_PI_Q = pd.DataFrame(PI_q_his).T


  



In [17]:
df_Q

A                          ...           C                        
               1           2           3  ...           1           2           3
0.00    7.427177    2.539811    4.234101  ...    7.258074    3.843987    4.228700
0.05    8.547555    3.322070    4.512047  ...    7.476628    4.613717    4.850772
0.10    8.895851    4.380142    5.057852  ...    8.214452    5.333476    5.268947
0.15    9.524300    5.199579    6.069211  ...    9.138125    6.126182    6.046186
0.20   10.312213    5.765740    6.714259  ...    9.287023    7.203204    6.511637
0.25   11.064952    6.914617    7.194415  ...   10.088877    8.141920    7.001846
0.30   12.111226    8.055273    7.761257  ...   10.997648    9.424033    8.273041
0.35   13.224748    9.731249    8.852220  ...   12.000980   10.961166    9.311260
0.40   15.023839   11.476057   10.296032  ...   14.176802   12.800307   10.366009
0.45   16.276764   13.255345   12.027278  ...   15.366218   14.517798   12.016972
0.50   18.065365   15.680276   13.287936  ...   17.020054   16.425976   13.841088
0.55   20.605415   18.514380   15.615096  ...   19.421141   19.142674   16.124843
0.60   23.773445   22.059822   18.694338  ...   22.473762   23.234112   19.415917
0.65   28.481859   26.192303   22.766475  ...   26.371902   27.547764   22.831894
0.70   33.562016   32.894618   29.219702  ...   32.497523   33.532975   28.892284
0.75   42.519469   41.278927   37.539684  ...   41.027804   43.745768   36.125331
0.80   55.210184   53.042591   50.262707  ...   53.311122   56.764259   48.931181
0.85   74.153563   75.561587   71.195786  ...   74.906730   77.066846   70.307476
0.90  118.205849  118.537542  114.793410  ...  117.872165  119.957488  114.043023
0.95  250.854817  253.456228  248.032079  ...  251.110201  254.054113  246.387334

[20 rows x 8 columns]

In [18]:
df_PI_Q

,A,B,C
0.00,1,1,1
0.05,1,1,1
0.10,1,1,1
0.15,1,1,1
0.20,1,1,1
0.25,1,2,1
0.30,1,2,1
0.35,1,2,1
0.40,1,2,1
0.45,1,2,1


Now we will implement TD($\lambda$) learning. (We will work with $\lambda$ = 0.1). We will use TD($\lambda$) for policy evaluation of the policy given in the question $[2, 2, 2]$.




I will use the algorithm mentioned on the webpage  (http://incompleteideas.net/book/ebook/node75.html)




In [ ]:
lr = 0.3 # Learning rate 
lmbd = 0.1

def TD_learning (beta):
  V = {}
  e = {}
  pi = {}
  #Initializing V and s and pi
  for s in States:
    V[s] = random.uniform(0, 1)
    e[s] = 0
    pi[s] = 2 #Questions asks to initialize with 2.
    
  
  # Policy evaluation part
  for i in range(200): #num of episodes
    alpha = lr * (0.8)** (i//20)
    S = random.choice(States)
    for j in range(500): #Since we don't have a termination state we go for finite steps.
      A = pi[S]
      (Sf, R) = take_action (S, A)
      dl = R + beta*V[Sf]-V[S]
      e[S] = e[S] + 1
      for s in States:
        V[s] = V[s] + alpha*dl*e[s]
        e[s] = beta*lmbd*e[s]

      S = Sf
      
    

  return V


In [ ]:
v_td = {}
# ptTD= {}

for b in range (0, 96, 5):
  beta = b /100
  # print(beta ,value_iteration(beta))
  v = TD_learning(beta)
  v_td[beta] =  v  
  # piItResults[beta] = pi



In [60]:
dfVtd = pd.DataFrame(v_td).T

dfVtd

,A,B,C
0.00,3.245897,15.595116,3.764823
0.05,3.401948,16.020231,4.537337
0.10,4.232484,16.069555,5.083271
0.15,4.773753,17.419229,6.154648
0.20,5.566269,18.387649,6.997948
0.25,7.065073,19.643256,8.242474
0.30,8.098521,20.607007,9.491314
0.35,9.110557,20.519571,10.158968
0.40,10.860666,23.726547,12.300443
0.45,13.007065,25.752240,14.019890
